# Creating a TorchModel
Base models built in PyTorch are already supported by LUME-model. We demonstrate how to create and execute a `TorchModel` below.

In [1]:
import torch

from lume_model.models import TorchModel, TorchModule
from lume_model.variables import ScalarInputVariable, ScalarOutputVariable

## Building a Model from Scratch
Instantiation of a `TorchModel` requires specification of the base model (`torch.nn.Module`) and in-/output variables.

In [2]:
# exemplary model definition
base_model = torch.nn.Sequential(
    torch.nn.Linear(2, 1),
)

# variable specification
input_variables = [
    ScalarInputVariable(name="input1", default=0.1, value_range=[0.0, 1.0]),
    ScalarInputVariable(name="input2", default=0.2, value_range=[0.0, 1.0]),
]
output_variables = [
    ScalarOutputVariable(name="output"),
]

# creation of TorchModel
example_model = TorchModel(
    model=base_model,
    input_variables=input_variables,
    output_variables=output_variables,
)

## Loading a Model from File
An already created model can be saved to a YAML file by calling the `dump` method. The model can then be loaded by simply passing the file to the constructor.

In [3]:
torch_model = TorchModel("../tests/test_files/california_regression/torch_model.yml")
print(torch_model.yaml())

model_class: TorchModel
input_variables:
  MedInc:
    variable_type: scalar
    default: 3.7857346534729004
    is_constant: false
    value_range: [0.4999000132083893, 15.000100135803223]
  HouseAge:
    variable_type: scalar
    default: 29.282135009765625
    is_constant: false
    value_range: [1.0, 52.0]
  AveRooms:
    variable_type: scalar
    default: 5.4074907302856445
    is_constant: false
    value_range: [0.8461538553237915, 141.90908813476562]
  AveBedrms:
    variable_type: scalar
    default: 1.1071722507476807
    is_constant: false
    value_range: [0.375, 34.06666564941406]
  Population:
    variable_type: scalar
    default: 1437.0687255859375
    is_constant: false
    value_range: [3.0, 28566.0]
  AveOccup:
    variable_type: scalar
    default: 3.035413980484009
    is_constant: false
    value_range: [0.692307710647583, 599.7142944335938]
  Latitude:
    variable_type: scalar
    default: 35.28323745727539
    is_constant: false
    value_range: [32.65999984741

## Model Execution and TorchModule
Calling the `evaluate` method allows for model execution on dictionary input. Additionally, instances of `TorchModel` can also be wrapped in a `TorchModule` which is a subclass of `torch.nn.Module`. This allows for seamless integration with `PyTorch` based packages like [BoTorch](https://botorch.org/) and [Xopt](https://christophermayes.github.io/Xopt/).

In [4]:
# generate exemplary input
input_dict = torch_model.random_input(n_samples=1)
input_dict

{'MedInc': tensor([11.2651]),
 'HouseAge': tensor([44.3406]),
 'AveRooms': tensor([130.5891]),
 'AveBedrms': tensor([19.3163]),
 'Population': tensor([11930.1680]),
 'AveOccup': tensor([212.5965]),
 'Latitude': tensor([37.1786]),
 'Longitude': tensor([-114.7374])}

In [5]:
# execute TorchModel
torch_model.evaluate(input_dict)

{'MedHouseVal': tensor(-2.4484, dtype=torch.float64)}

In [6]:
# wrap in TorchModule
torch_module = TorchModule(model=torch_model)

In [7]:
# execute TorchModule
input_tensor = torch.tensor([input_dict[k] for k in torch_module.input_order]).unsqueeze(0)
torch_module(input_tensor)

tensor(-2.4484, dtype=torch.float64)